<a href="https://colab.research.google.com/github/gubrx/Deep-learning-eBSDE/blob/main/Forward_deep_ebsde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drive_import

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import time
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

from scipy.integrate import quad
import scipy
import scipy.optimize as opt
import pandas as pd
import os
import pickle


# Networks

In [12]:
#Neural Network
class Net( tf.keras.Model):
    def __init__(self, cond_lambd, lambda_lim, dim, nbNeurons, activation= tf.nn.tanh):
        super().__init__()
        self.nbNeurons = nbNeurons
        self.dim = dim
        self.ListOfDense = [layers.Dense( nbNeurons[i],activation= activation, kernel_initializer= tf.keras.initializers.GlorotNormal())  for i in range(len(nbNeurons)) ] +[layers.Dense(self.dim, activation= None, kernel_initializer= tf.keras.initializers.GlorotNormal())]
        if cond_lambd:
          self.lambd= tf.Variable(tf.keras.initializers.GlorotNormal()([1]),  trainable = True, dtype=tf.float32, constraint=lambda t: tf.clip_by_value(t, -lambda_lim, lambda_lim))

    def call(self,inputs):
        x = inputs
        for layer in self.ListOfDense:
            x = layer(x)
        return x

# StochasticFactor

In [4]:
class StochasticFactor():
    def __init__(self, x0, dt, T_H, dim):
        self.x0 = x0
        self.dt = dt
        self.T_H = T_H
        self.dim = dim

    def one_step(self, V, mu, kappa):
        dw_sample = np.sqrt(self.dt)*np.random.normal(size=(len(V), self.dim))
        V = V + mu(V)*self.dt + np.sum(np.array(kappa)*dw_sample, axis=1)
        return dw_sample, V

    def sample(self, mu, kappa, num_sample):
        V = np.ones((1, num_sample)) * self.x0
        dW = np.empty((0, num_sample, self.dim))
        T_A = np.zeros(num_sample) + np.inf
        indT_H = int(np.ceil(self.T_H/self.dt))

        for i in range(indT_H):
            dw_sample, V_new = self.one_step(V[-1, :], mu, kappa)
            V = np.vstack([V, V_new[None, :]]) # Stack vertically
            dW = np.vstack([dW, dw_sample[None, :]]) # Adjusted for extra dimension

        Lstart = V[-1, :] - self.x0*np.ones(num_sample)
        i = indT_H+1  # Start from the index of T_H in the time grid
        while np.any(T_A == np.inf) and i < 20/self.dt:
            dw_sample, V_new = self.one_step(V[-1, :], mu, kappa)
            V = np.vstack([V, V_new[None, :]])
            dW = np.vstack([dW, dw_sample[None, :]])

            L = V_new - self.x0
            condition = L * Lstart <= 0
            # Update T_A for indices where condition is met and T_A not changed yet
            for j in np.where((condition) & (T_A == np.inf))[0]:
                T_A[j] = i

            i += 1

        if i == 20 * indT_H:
            print("The maximum return time over num_sample is large, greater than T=20. Consider changing parameters of the stochastic factor.")

        return np.array(T_A, int), dW, V


    def sol_SDE(self, X, dW, drift, vol, tinit, gammainit):
        x_sample = np.zeros([len(X[:,0]), len(X[0,:])])
        x_sample[tinit, :] = np.ones(len(X[0,:])) * gammainit
        for i in range(tinit, len(X[:,0])-1):
          x_sample[i+1, :] = x_sample[i, :] + x_sample[i, :]*drift(X[i, :])*self.dt \
            + x_sample[i,:] * tf.reduce_sum(vol(X[i, :]) * dW[i, :], axis=1)

        return x_sample


class OrnsteinUhlenbeck(StochasticFactor):
    def __init__(self, x0, dt, T_H, dim, muval, nu, kappa):
        super().__init__(x0, dt, T_H, dim)
        self.muval = muval
        self.kappa = kappa
        self.x0 = x0
        self.nu = nu

    def mu(self, x):
      return -self.muval * (x - self.nu)





# ErgodicFactorModel

In [5]:
class ErgodicFactorModel():
  def __init__(self, stochastic_factor):
    self.stochastic_factor = stochastic_factor
    self.dt = stochastic_factor.dt
    self.T_H = stochastic_factor.T_H
    self.dim = stochastic_factor.dim

  def proj_Pi(self, Pi, X):
      Pi = tf.convert_to_tensor(Pi, dtype=X.dtype)
      # Ensure Pi has shape (dim, 2)
      assert Pi.shape[1] == 2, "Pi should have shape (dim, 2)"

      Pi_expanded = tf.expand_dims(tf.expand_dims(Pi, 0), 0)

      X_expanded = tf.expand_dims(X, -1)
      lower_mask = X_expanded < Pi_expanded[..., 0:1]
      upper_mask = X_expanded > Pi_expanded[..., 1:2]

      X_proj = tf.where(lower_mask, Pi_expanded[..., 0:1], X_expanded)
      X_proj = tf.where(upper_mask, Pi_expanded[..., 1:2], X_proj)

      X_proj = tf.squeeze(X_proj, -1)

      return X_proj

  # forward scheme eBSDE
  def forward_BSDE(self, T_A, dW, V, ergodic_model, kerasModel):
      nbr_traj = len(V[0,:])
      maxind = max(T_A)
      dW, V = tf.convert_to_tensor(dW, dtype=tf.float32), tf.convert_to_tensor(V, dtype=tf.float32)
      Y_sim = np.zeros([maxind+1, nbr_traj])
      Z_sim = np.zeros((maxind, nbr_traj, self.dim))

      Y_trans = ergodic_model.Y0 * tf.ones([nbr_traj], dtype=tf.float32)
      Y_sim[0, :] = Y_trans.numpy()
      for iStep in range(maxind):
          Z = kerasModel(tf.expand_dims(V[iStep, :nbr_traj], axis=-1))
          Y_trans = Y_trans - self.dt * tf.transpose(ergodic_model.f(V[iStep, :nbr_traj], Z)) + kerasModel.lambd.numpy() * tf.ones([nbr_traj], dtype=tf.float32) * self.dt + tf.reduce_sum(Z * dW[iStep, :], axis=1)
          Y_sim[iStep+1, :] = Y_trans.numpy()
          Z_sim[iStep, :] = Z.numpy()

      return Y_sim, Z_sim

  def mean_abs_error(self, T_A, Y_ex, Y_sim):
      indTH = int(np.ceil(self.T_H / self.dt))
      nbr_traj = Y_ex.shape[1]
      Y_ex_mod = Y_ex[:indTH+1, :nbr_traj]
      mask_rel = tf.cast(tf.math.abs(Y_ex_mod) > 0, dtype=tf.float32)
      Y_sim_mod = Y_sim[:indTH+1, :nbr_traj]
      # Absolute error
      Etot = np.abs(Y_ex_mod - Y_sim_mod)
      # Relative error
      Etotrel = np.where(Y_ex_mod != 0, np.abs((Y_ex_mod - Y_sim_mod) / Y_ex_mod), 0)

      Emoy = np.sum(Etot, axis=1) / nbr_traj
      Emoy_rel = np.sum(Etotrel, axis=1) / np.sum(mask_rel, axis=1)

      return Emoy, Emoy_rel

  def integral_error(self, T_A, Y_ex, Y_sim, Z_ex, Z_sim):
      indTH = int(np.ceil(self.T_H / self.dt))
      nbr_traj = Y_ex.shape[1]
      interrY = tf.reduce_mean(self.dt * tf.reduce_sum(abs(Y_ex[:indTH, :nbr_traj] - Y_sim[:indTH, :nbr_traj]), axis=0))
      interrZ = tf.reduce_mean(self.dt * tf.reduce_sum(tf.norm(Z_ex[:indTH, :nbr_traj, :] - Z_sim[:indTH, :nbr_traj, :], axis=-1)**2, axis=0))

      return interrY, interrZ

class EV(ErgodicFactorModel):
  def __init__(self, stochastic_factor):
      super().__init__(stochastic_factor)

  # Approximation Monte Carlo lambda for generators depending only on the stochastic factor V
  def lambdapprox(self, V, T_A):
      T_A = tf.convert_to_tensor(T_A, dtype=tf.float32)
      V = tf.convert_to_tensor(V, dtype=tf.float32)
      T_Aint = tf.cast(T_A, dtype=tf.int32)

      timestep_range = tf.range(tf.shape(V)[0])[:, tf.newaxis]
      selec_index = timestep_range <= T_Aint

      V_selec = tf.where(selec_index, V, tf.zeros_like(V))
      f_values = self.f(V_selec, 0)
      appint_values = tf.reduce_sum(f_values * self.stochastic_factor.dt, axis=0)

      meanappint = tf.reduce_sum(appint_values)
      meanT_R = tf.reduce_sum(T_A * self.dt)

      return float(meanappint / meanT_R)

# The solutions for Example 1 and Example 2 are valid for a stochastic factor of type OrnsteinUhlenbeck and in dimension 1.
class Example1(EV):
  def __init__(self, ornstein_uhlenbeck, Cv):
      super().__init__(ornstein_uhlenbeck)
      self.Cv = Cv
      self.muval = ornstein_uhlenbeck.muval
      self.kappa = ornstein_uhlenbeck.kappa
      self.Y0 = (self.Cv/(self.muval + (1/2)*self.kappa[0]**2))*np.sqrt(2*np.pi)*scipy.stats.norm.cdf(ornstein_uhlenbeck.x0, 0, 1)
      self.Zlim = tf.norm(tf.constant(self.kappa, dtype=tf.float32))*(self.Cv / (self.muval - self.Cv))
      self.lambdlim = self.Cv*tf.exp(-1**2/2)
      self.lambd_ex = 0

  # Driver
  def f(self, v, z):
      return v*self.Cv*tf.exp(-v**2/2)

  # Exact solution
  def y_ex(self, V):
      return (self.Cv/(self.muval + (1/2)*self.kappa[0]**2))*np.sqrt(2*np.pi)*scipy.stats.norm.cdf(V, 0, 1)

  def z_ex(self, V):
      return (self.Cv/(self.muval + (1/2)*self.kappa[0]**2))*tf.exp(-V**2/2)


class Example2(EV):
  def __init__(self, ornstein_uhlenbeck, Cv):
      super().__init__(ornstein_uhlenbeck)
      self.Cv = Cv
      self.dt = ornstein_uhlenbeck.dt
      self.muval = ornstein_uhlenbeck.muval
      self.kappa = np.sqrt(2*self.muval) #kappa fixed from drift, see Annex A
      self.x0 = ornstein_uhlenbeck.x0

      self.A1 = self.Cv/(self.kappa**2)
      self.A2 = 2*self.A1
      self.Y0 = 1+quad(lambda y: np.exp(y**2/2)*(self.A1*np.exp(-y**2) + self.A2*(scipy.stats.norm.cdf(y, 0, 1) - 1)), 0, self.x0)[0]

      self.Zlim = tf.norm(self.kappa)*(self.Cv / (self.muval - self.Cv))
      self.lambd_ex = self.Cv / np.sqrt(2*np.pi)
      self.lambdlim = self.Cv*tf.exp(-1**2/2)

  def integrandpos(self, y):
      return np.exp(y**2/2)*(self.A1*np.exp(-y**2) + self.A2*(scipy.stats.norm.cdf(y, 0, 1) - 1))

  def integrandneg(self, y):
      return np.exp(y**2/2)*(-self.A1*np.exp(-y**2) + self.A2*scipy.stats.norm.cdf(y, 0, 1))

  # Exact solution
  def y_ex(self, T_A, X):
      nbr_time_step, nbr_traj = X.shape
      Y_ex = np.zeros((nbr_time_step, nbr_traj))
      for iStep in range(nbr_time_step):
          listM = tf.cast(iStep <= T_A, tf.float32)
          Neg = tf.cast(X[iStep, :] < 0, tf.float32)
          Pos = tf.cast(X[iStep, :] >= 0, tf.float32)
          Xpos = X[iStep, :] * Pos * listM
          Xneg = X[iStep, :] * Neg * listM
          for m in range(nbr_traj):
              if listM[m]:  # Only integrate for active trajectories
                  if Pos[m]:
                      Y_ex[iStep, m] = 1+ quad(self.integrandpos, 0, Xpos[m])[0]
                  if Neg[m]:
                      Y_ex[iStep, m] = 1+quad(self.integrandneg, 0, Xneg[m])[0]

      return Y_ex

  def z_ex(self, X):
    Pos = X >= 0
    Neg = X < 0
    return Neg*self.integrandneg(X) + Pos * self.integrandpos(X)

  # Driver
  def f(self, v, z):
      return self.Cv*abs(v)*tf.exp(-v**2/2)


class ErgodicPowerSE(ErgodicFactorModel):
  def __init__(self, stochastic_factor, market_price, p, b, delt):
      super().__init__(stochastic_factor)
      self.delt = delt #risk aversion
      self.gamma = 1 /(1 - self.delt)
      self.Cv = (1/2)*(delt/(1-delt))*np.linalg.norm(p)*np.max([2, b])
      self.lambdlim = (1/2)*(delt/(1-delt))*b**2
      self.dt = stochastic_factor.dt
      self.mu = stochastic_factor.mu
      self.kappa = stochastic_factor.kappa
      self.Zlim = tf.norm(tf.constant(self.kappa, dtype=tf.float32))*(self.Cv / (self.stochastic_factor.muval - self.Cv))
      self.Y0 = 0.
      self.market_price = market_price
      self.lambd_ex = 'Not known'

  def thet(self, x):
    return self.market_price(x)

  def beta(self, V, l):
    return (self.delt / (1 - self.delt)) * (self.gamma / 2) * tf.norm(self.thet(V), axis=-1)**2 - self.gamma * l

  def nu(self, V):
    return (self.delt / (1 - self.delt)) * self.thet(V)

  # Approximation Monte Carlo - Power utility example
  def loss(self, T_A, X, dW, l):
    Nbsimul = len(X[0, :])
    G = self.stochastic_factor.sol_SDE(X, dW, lambda x: self.beta(x, l), self.nu, 0, 1)
    selected_G = tf.gather_nd(G, tf.stack([T_A, tf.range(Nbsimul)], axis=1))
    lossval = tf.abs(tf.reduce_mean(selected_G) - 1)

    return lossval.numpy()

  def lambda_app_newt(self, num_sample):
    T_A, dW, V = self.stochastic_factor.sample(self.mu, self.kappa, num_sample)
    return scipy.optimize.fsolve(lambda l: self.loss(T_A, V, dW, l), 0)

  # Driver
  def f(self, v, z):
    return (1/2)*self.delt/(1 - self.delt)*tf.norm(z + self.thet(v), axis = -1)**2 + (1/2)*tf.norm(z, axis=-1)**2

  # Exact solution
  def yex(self, EC):
    return self.Y0 + np.reshape((1/self.gamma)*np.log(EC),  (len(EC[:,0]), len(EC[0, :])))

  # optimal portoflio
  def optimal_strategy(self, Pi, V, kerasModel):
    Ztot = np.zeros((len(V[:,0]), len(V[0,:]), len(Pi) ))
    thetV = np.zeros((len(V[:,0]), len(V[0,:]), len(Pi) ))
    for iStep in range(len(V[:,0])):
      Z = kerasModel(tf.expand_dims(V[iStep, :len(V[0,:])], axis=-1))
      Ztot[iStep, :] = Z.numpy()
      thetV[iStep,:] = self.market_price(V[iStep,:])

    return self.proj_Pi(Pi, (1/(1-self.delt))*(thetV + Ztot))


class ErgodicPowerGen(ErgodicFactorModel):
  def __init__(self, stochastic_factor, market_price, p, b, delt):
      super().__init__(stochastic_factor)
      self.delt = delt #risk aversion
      self.gamma = 1 /(1 - self.delt)
      self.Cv = (1/2)*(delt/(1-delt))*np.linalg.norm(p)*np.max([2, b])
      self.lambdlim = (1/2)*(delt/(1-delt))*b**2
      self.dt = stochastic_factor.dt
      self.mu = stochastic_factor.mu
      self.muval = self.stochastic_factor.muval
      self.kappa = stochastic_factor.kappa
      self.Zlim = tf.norm(tf.constant(self.kappa, dtype=tf.float32))*(self.Cv / (self.muval - self.Cv))
      self.market_price = market_price
      self.Y0 = 0
      self.lambd_ex = 'Not known'

  def thet(self, x):
    return self.market_price(x)

  # Driver
  def f(self, v, z):
    return (1/2)*self.delt/(1 - self.delt)*(z[:,0] + self.thet(v)[:,0])**2 + (1/2)*tf.norm(z, axis=-1)**2

  # Optimal portoflio
  def optimal_strategy(self, Pi, V, kerasModel):
    Ztot = np.zeros((len(V[:,0]), len(V[0,:]), len(Pi) ))
    thetV = np.zeros((len(V[:,0]), len(V[0,:]), len(Pi) ))
    for iStep in range(len(V[:,0])):
      Z = kerasModel(tf.expand_dims(V[iStep, :len(V[0,:])], axis=-1))
      Ztot[iStep, :] = Z.numpy()
      thetV[iStep,:] = self.market_price(V[iStep,:])

    return self.proj_Pi(Pi, (1/(1-self.delt))*(thetV + Ztot))







# Solvers_eBSDE

In [6]:
class SolverBase:
    # mathModel          Math model
    # modelKeras         Keras model
    # lRate              Learning rate
    def __init__(self, ErgodicFactorModel, lRate):
        self.ErgodicFactorModel = ErgodicFactorModel
        self.StochasticFactor = ErgodicFactorModel.stochastic_factor
        self.lRate = lRate

    @tf.function
    def proj(self, Z, Zlim):
        norms = tf.norm(Z, axis=1, keepdims=True)
        # Calculate scaling factors for rows where the norm exceeds Zlim
        scaling_factors = tf.where(norms > Zlim, Zlim / norms, 1)
        Z_proj = Z * scaling_factors

        return Z_proj

# Global solver
class SolverGlobaleBSDE(SolverBase):
    def __init__(self, ErgodicFactorModel, modelKerasUZ , lRate):
        super().__init__(ErgodicFactorModel, lRate)
        self.modelKerasUZ = modelKerasUZ

    def train(self, batchSize, batchSizeVal, num_epoch, num_epochExt):
        @tf.function
        def optimizeBSDE(nbSimul):
            T_A, dW, V = self.StochasticFactor.sample(self.StochasticFactor.mu, self.StochasticFactor.kappa, nbSimul)
            dW, V = tf.convert_to_tensor(dW, dtype=tf.float32), tf.convert_to_tensor(V, dtype=tf.float32)
            Y = []
            maxind = max(T_A)
            Y_trans = self.ErgodicFactorModel.Y0 * tf.ones([nbSimul], dtype=tf.float32)
            for iStep in range(int(maxind)):
              input_tensor = tf.expand_dims(V[iStep, :], axis=-1)
              Z = self.modelKerasUZ(input_tensor)
              Y_trans = Y_trans - self.StochasticFactor.dt * self.ErgodicFactorModel.f(V[iStep, :], Z) + self.modelKerasUZ.lambd * tf.ones([nbSimul], dtype=tf.float32) * self.StochasticFactor.dt + tf.reduce_sum(Z * dW[iStep, :], axis=1)
              indices = tf.where(T_A == iStep)[:, 0]
              Y_trans_selected = tf.gather(Y_trans, indices)
              Y.append(Y_trans_selected)

            Y = tf.concat(Y, axis=0)

            return tf.reduce_mean(tf.square(Y - self.ErgodicFactorModel.Y0))

        # train to optimize control
        @tf.function
        def trainOptNN(nbSimul, optimizer):
            with tf.GradientTape() as tape:
                objFunc_Z = optimizeBSDE(nbSimul)
            gradients = tape.gradient(objFunc_Z, self.modelKerasUZ.trainable_variables)
            optimizer.apply_gradients(zip(gradients, self.modelKerasUZ.trainable_variables))
            return objFunc_Z

        optimizerN = optimizers.Adam(learning_rate = self.lRate)

        self.listlambd = []
        self.lossList = []
        self.duration = 0
        for iout in range(num_epochExt):
            start_time = time.time()
            for epoch in range(num_epoch):
                # un pas de gradient stochastique
                trainOptNN(batchSize, optimizerN)
            end_time = time.time()
            rtime = end_time-start_time
            self.duration += rtime
            objError_Yterm = optimizeBSDE(batchSizeVal)
            lambd = self.modelKerasUZ.lambd.numpy()
            print(" Error",objError_Yterm.numpy(),  " elapsed time %5.3f s" % self.duration, "lambda so far ",lambd, 'epoch', iout)
            self.listlambd.append(lambd)
            self.lossList.append(objError_Yterm)

        return self.listlambd, self.lossList

# Local solver
class SolverLocaleBSDE(SolverBase):
    def __init__(self, ErgodicFactorModel, modelKerasY, modelKerasZ, lRate):
        super().__init__(ErgodicFactorModel, lRate)
        self.modelKerasY = modelKerasY
        self.modelKerasZ = modelKerasZ

    def train(self, batchSize, batchSizeVal, num_epoch, num_epochExt):
        @tf.function
        def optimizeBSDE(nbSimul):
            T_A, dW, V = self.StochasticFactor.sample(self.StochasticFactor.mu, self.StochasticFactor.kappa, nbSimul)
            dW, V = tf.convert_to_tensor(dW, dtype=tf.float32), tf.convert_to_tensor(V, dtype=tf.float32)
            maxind = max(T_A)
            Loss = []
            loss_term = tf.zeros([nbSimul], dtype=tf.float32)
            Y_trans = self.ErgodicFactorModel.Y0 * tf.ones([nbSimul], dtype=tf.float32)
            for iStep in range(int(maxind)):
              input_tensor = tf.expand_dims(V[iStep, :], axis=-1)
              Z = self.modelKerasZ(input_tensor)
              Y = self.modelKerasY(input_tensor)
              toAdd = self.StochasticFactor.dt * self.ErgodicFactorModel.f(V[iStep, :], Z) - self.modelKerasZ.lambd * tf.ones([nbSimul], dtype=tf.float32) * self.StochasticFactor.dt - tf.reduce_sum(Z * dW[iStep, :], axis=1)
              loss_term = loss_term + toAdd

              indices = tf.where(T_A >= iStep)[:, 0]
              loss_term_selec = tf.expand_dims(tf.gather(loss_term, indices), axis=-1)
              Y_selec = tf.gather(Y, indices)

              Loss.append(Y_selec  + loss_term_selec - self.ErgodicFactorModel.Y0*tf.ones_like(loss_term_selec))

            Loss = tf.concat(Loss, axis=0)

            return tf.reduce_mean(tf.square(Loss))

        # train to optimize control
        @tf.function
        def trainOptNN(nbSimul, optimizerZ, optimizerY):
            with tf.GradientTape(persistent=True) as tape:
                objFunc = optimizeBSDE(nbSimul)
            gradientsZ = tape.gradient(objFunc, self.modelKerasZ.trainable_variables)
            gradientsY = tape.gradient(objFunc, self.modelKerasY.trainable_variables)

            optimizerZ.apply_gradients(zip(gradientsZ, self.modelKerasZ.trainable_variables))
            optimizerY.apply_gradients(zip(gradientsY, self.modelKerasY.trainable_variables))

            del tape
            return objFunc

        optimizerZ = optimizers.Adam(learning_rate=self.lRate)
        optimizerY = optimizers.Adam(learning_rate=self.lRate)

        self.listlambd = []
        self.lossList = []
        self.duration = 0
        for iout in range(num_epochExt):
            start_time = time.time()
            for epoch in range(num_epoch):
                # un pas de gradient stochastique
                trainOptNN(batchSize, optimizerZ, optimizerY)
            end_time = time.time()
            rtime = end_time-start_time
            self.duration += rtime
            objError_Yterm = optimizeBSDE(batchSizeVal)
            lambd = self.modelKerasZ.lambd.numpy()
            print(" Error",objError_Yterm.numpy(),  " elapsed time %5.3f s" % self.duration, "lambda so far ",lambd, 'epoch', iout)
            self.listlambd.append(lambd)
            self.lossList.append(objError_Yterm)


        return self.listlambd, self.lossList

# Main training

In [ ]:
# Parameters
T_H = 1
h = 0.01
kappa = [0.8]
dim = len(kappa)
nu = 0
x0 = 0
Mlambd = 100000

# Power utility examples parameters
delt = 0.5  # risk aversion
p = [0.8]  # Lipschitz constant market price of risk - of dimension dim
b = 3  # born market price of risk
Pi = [[-np.inf, np.inf], [0, 0]]

def market_price(v):
    v = tf.convert_to_tensor(v, dtype=tf.float32)
    pa = tf.convert_to_tensor(p, dtype=tf.float32)
    v = tf.reshape(v, (-1, 1))
    pv = pa * v

    norm_pv = tf.norm(pv, axis=1, keepdims=True)
    condition = norm_pv > b
    pv = tf.where(condition, (pv / norm_pv) * b, pv)

    return pv

# NN Parameters
nbNeuron = 20 + dim
nbLayer = 2
num_epochExt = 100
num_epoch = 100
batchSize = 64
lRate = 0.0003
activation = tf.nn.tanh

# Select example
example_name = input("Enter the example to run (Example1, Example2, ErgodicPowerSE, ErgodicPowerGen): ")
if example_name in ['Example1', 'Example2']:
    Cv = float(input("Enter the value for 'Cv': "))
else:
    Cv = (1 / 2) * (delt / (1 - delt)) * np.linalg.norm(p) * np.max([2, b])
    print('Cv =', Cv)

# Prompt for a valid value of muval
muval = None
while muval is None or muval <= Cv:
    try:
        muval_input = input(f"Enter a value for 'muval' greater than {Cv}: ")
        muval = float(muval_input)
        if muval <= Cv:
            print(f"The value of 'muval' must be greater than {Cv}. You entered: {muval}")
    except ValueError:
        print("Please enter a valid numeric value.")

print(f"'muval' set to: {muval}")

# Initialize the selected example
if example_name == 'Example2':
    kappa = [np.sqrt(2*muval)]

solver_name = input("Enter the solver to run (Global, Local): ")

stochastic_factor = OrnsteinUhlenbeck(x0, h, T_H, dim, muval, nu, kappa)
if example_name == 'Example1':
    example = Example1(stochastic_factor, Cv)
elif example_name == 'Example2':
    kappa = [np.sqrt(2*muval)]
    example = Example2(stochastic_factor, Cv)
elif example_name == 'ErgodicPowerSE':
    stochastic_factor = OrnsteinUhlenbeck(x0, h, T_H, dim, muval, nu, kappa)
    example = ErgodicPowerSE(stochastic_factor, market_price, p, b, delt)
elif example_name == 'ErgodicPowerGen':
    stochastic_factor = OrnsteinUhlenbeck(x0, h, T_H, dim, muval, nu, kappa)
    example = ErgodicPowerGen(stochastic_factor, market_price, p, b, delt)
else:
    raise ValueError("Invalid example name.")

# Some values
lambda_lim = example.lambdlim
if hasattr(example, 'lambd_ex'):
    print(f'lambda exact= {example.lambd_ex}')

# Neural network
layerSize = nbNeuron * np.ones((nbLayer,), dtype=np.int32)
if solver_name == 'Global':
  kerasModel = Net(True, lambda_lim, dim, layerSize, activation)
  solver = SolverGlobaleBSDE(example, kerasModel, lRate)
if solver_name == 'Local':
  kerasModelY = Net(False, lambda_lim, 1, layerSize, activation)
  kerasModelZ = Net(True, lambda_lim, dim, layerSize, activation)
  solver = SolverLocaleBSDE(example, kerasModelY, kerasModelZ, lRate)

# train and  get solution
lambdlist, lossT_Hlist = solver.train(batchSize, batchSize * 100, num_epoch, num_epochExt)

###### PLOTS ######
if example_name =='Example1' or example_name == 'Example2':
  lambda_benchmark = example.lambd_ex
  lambd_label = r'$\lambda$ exact'

if example_name == 'ErgodicPowerSE':
  lambd_newt_values = []
  for i in range(3):
      lambd_newt = example.lambda_app_newt(Mlambd)
      lambd_newt_values.append(lambd_newt)

  std_lambd_newt = np.sqrt(np.var(lambd_newt_values))
  lambda_benchmark = np.mean(lambd_newt_values)
  lambd_label = r'$\hat{\lambda}$'
  print('Mean lambda MC method =', lambda_benchmark)
  print('Standard deviation lambda MC method =', std_lambd_newt)

if example_name == 'ErgodicPowerGen':
  lambda_benchmark = 'Not known'

if solver_name == 'Global':
  loss_name = r'$L^{B_{\epsilon}}(\theta, \bar{\lambda})$'
  solver_label = 'GeBSDE'

if solver_name == 'Local':
  loss_name = r'$L_{loc}^{B_{\epsilon}}(\theta_{1}, \theta_{2}, \bar{\lambda})$'
  solver_label = 'LAeBSDE'

Nepoch = range(0, num_epoch*num_epochExt, num_epoch)
plt.figure()
plt.plot(Nepoch, lossT_Hlist, label=f"{loss_name} - {solver_label}")
plt.grid(True, which = 'both', linestyle='--', alpha=0.7)
plt.legend()
plt.xlabel('Number of Epochs')
plt.tight_layout()
plt.title('Loss training result')
plt.show()

plt.figure()
plt.plot(Nepoch, lambdlist, label=rf'$\bar{{\lambda}}$ - {solver_label}')
if lambda_benchmark != 'Not known':
    plt.plot(Nepoch, lambda_benchmark * np.ones(len(Nepoch)), "r--", label=lambd_label)
plt.grid(True, which = 'both', linestyle='--', alpha=0.7)
plt.legend()
plt.xlabel('Number of Epochs')
plt.tight_layout()
plt.title('Convergence lambda training result')
plt.show()



Enter the example to run (Example1, Example2, ErgodicPowerSE, ErgodicPowerGen): Example1
Enter the value for 'Cv': 1
Enter a value for 'muval' greater than 1.0: 2
'muval' set to: 2.0
Enter the solver to run (Global, Local): Local
lambda exact= 0
 Error 0.33045068  elapsed time 436.114 s lambda so far  [0.4640553] epoch 0
 Error 0.080516644  elapsed time 444.316 s lambda so far  [0.4488413] epoch 1
 Error 0.05699098  elapsed time 450.964 s lambda so far  [0.4416721] epoch 2
 Error 0.054124672  elapsed time 457.983 s lambda so far  [0.43461055] epoch 3
 Error 0.052743465  elapsed time 466.251 s lambda so far  [0.42659858] epoch 4
 Error 0.051084016  elapsed time 476.686 s lambda so far  [0.4177617] epoch 5
 Error 0.049055457  elapsed time 483.681 s lambda so far  [0.40820152] epoch 6
 Error 0.046804927  elapsed time 499.593 s lambda so far  [0.39800632] epoch 7
 Error 0.044441912  elapsed time 511.005 s lambda so far  [0.3872546] epoch 8
 Error 0.042025153  elapsed time 518.171 s lambda 